In [20]:
#installing packages in JupyterLab temporarily
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install bs4
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install datetime
!{sys.executable} -m pip install selenium

**Packages**

In [113]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as BS
from selenium import webdriver
from datetime import datetime
import pandas as pd
import datetime
import getpass
import time
import re

**All pages with urls scraping**

In [122]:
############################################################################
# Downloading offers' urls and titles from the website controlled by Selenium
url = 'https://www.otomoto.pl/osobowe/'

options = webdriver.firefox.options.Options()
options.headless = False

driver = webdriver.Firefox(executable_path=r'C:\Users\Admin\Downloads\geckodriver-v0.29.0-win64\geckodriver.exe')
driver.get(url)
# create BS object from the page source
bs = BS(driver.page_source, 'html.parser')

#### click accept COOKIEs button
# waiting a few seconds to be sure that page will be fully loaded
time.sleep(4)
button = driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]').click()

# Extracting number of pages with offers list to iterate
maxSites = bs.find_all('span', {'class':'page'})[-1].text
print("Max site number: " + maxSites)

maxSites = 3 #zmienione do testowania

# All offers' titles and urls scraping
url_list = []
title_list = []

for i in range(int(maxSites)):
    if(i > 0):
        button = driver.find_element_by_xpath('//span[@class="icon-arrow_right"]').click()

    time.sleep(5)
    bs = BS(driver.page_source, 'html.parser')

    titles = bs.find_all('a', {'class':'offer-title__link'}) 

    links = [title['href'] for title in titles]
    
    titles_text = [title.text.strip() for title in titles]
 
    for link in links:
        url_list.append(link)
    for title in titles_text:
        title_list.append(title)
        
    print('loop:  ', i)
    
# Close browser:
driver.quit()

Max site number: 500
loop:   0
loop:   1
loop:   2


In [154]:
print(title_list[1])
print("#############")
print(url_list[1])

Hyundai i20 1.25
#############
https://www.otomoto.pl/oferta/hyundai-i20-model-2015-klima-tempomat-ledy-100-oryg-przebieg-86tys-ID6DFqDw.html#7ce9f71ee3


In [147]:
# Save urls to df
urls_df = pd.DataFrame({'offer_title': title_list, 'link': url_list})
urls_df.to_csv('urls.csv', index=False, encoding="Windows-1252")

In [110]:
headerOne = {'User-Agent': 'Mozilla/5.0'}



In [ ]:
cars = pd.DataFrame({'title':[], 'author':[], 'publisher':[], 'category':[], 
     'score':[], 'ebookPrice':[], 'audiobookPrice':[], 'paperPrice':[], 'peopleInterested':[]})

i = 0
for offer_url in url_list:
    site = offer_url
    print('Loop for idividual sites iterator ', i)
    req = Request(site, headers=headerOne)
    try:
        html = urlopen(req)
        bs = BS(html.read(), 'html.parser')
    except:
        continue

    time.sleep(1) 
    try:
        title = bs.find('h1', {'class':'title-text'}).text #wymaga poprawki, żeby obciąć ebook i audiobook
        title = re.split(' ebook', title)[0] #save only title when ebook type
        title = re.split(' audiobook', title)[0] #save only title when audiobook type
        title = re.split(' książka papierowa', title)[0] #save only title when paperback type
    except:
        title = ''

    try:
        author = bs.find('a', {'class': 'author-link'}).text
    except:
        author = ''

    try:
        publisher = bs.find('a', {'class':'category-link'}).text
    except:
        publisher = ''

    try:
        category = bs.find_all('a', {'class':'category-link'})[1].text
    except:
        category = ''

    try:
        score = bs.find('span', {'class':'votes-count'}).text
        score = re.findall('[0-5]{1}\,[0-9]', score)[0]
    except:
        score = ''

    try:
        ebookPrice = bs.find_all('h4',string = 'ebook')[-1].parent.next_sibling.text
        ebookPrice = re.findall('[0-9]+\,[0-9]+', ebookPrice)[0]
    except:
        ebookPrice = ''

    try:
        audiobookPrice = bs.find_all('h4',string = 'audiobook')[-1].parent.next_sibling.text
        audiobookPrice = re.findall('[0-9]+\,[0-9]+', audiobookPrice)[0]
    except:
        audiobookPrice = ''

    try:
        paperPrice = bs.find_all('h4',string = 'książka papierowa')[-1].parent.next_sibling.text
        paperPrice = re.findall('[0-9]+\,[0-9]+', paperPrice)[0]
    except:
        paperPrice = ''

    try:
        peopleInterested = bs.find('p', {'class':'readers-count-text'}).text
        peopleInterested = re.findall('[0-9]+', peopleInterested)[0]
    except:
        peopleInterested = ''

    i = i+1 
    
    book = {}

    book = {'title':title, 'author':author, 'publisher':publisher, 'category':category, 
             'score':score, 'ebookPrice':ebookPrice, 
            'audiobookPrice':audiobookPrice, 'paperPrice':paperPrice, 'peopleInterested':peopleInterested}

    books = books.append(book, ignore_index = True)

    books.to_csv('kryminały_resultBS.csv') #save each time to omit problems

    # print(cars)


In [213]:
###Single site test
from urllib import request
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as BS
import pandas as pd
import re
from datetime import datetime

headerOne = {'User-Agent': 'Mozilla/5.0'}

books = pd.DataFrame({'brand':[], 'model':[], 'version':[], 'generation':[], 'seller':[], 'year_production':[], 
                      'mileage':[], 'engine_capacity':[], 'fuel_type':[], 'power':[], 'gearbox':[],
                     'drive_wheel':[], 'CO2_emission':[], 'vehicle_type':[]})

site = 'https://www.otomoto.pl/oferta/volvo-xc-60-piekne-xc60-r-design-165-tys-km-serwis-orginal-xenon-skora-full-ID6DEw6n.html#7ce9f71ee3'

req = Request(site, headers=headerOne)

html = urlopen(req)
bs = BS(html.read(), 'html.parser')

try:
    brand = bs.find('span', text='Marka pojazdu').parent.a.text.strip()
except:
    brand = ''

print('This is the brand: ', brand)

try:
    model = bs.find('span', text='Model pojazdu').parent.a.text.strip()
except:
    model = ''

print('This is the model: ', model)

try:
    version = bs.find('span', text='Wersja').parent.a.text.strip()
except:
    version = ''

print('This is the version: ', version)

try:
    generation = bs.find('span', text='Generacja').parent.a.text.strip()
    #generation = language.find_all('li')[2].span.next_sibling.text
except:
    generation = ''

print('This is the generation: ', generation)

try:
    year_production = bs.find('span', text='Rok produkcji').parent.div.text.strip()
except:
    year_production = ''

print('This is the year_production: ', year_production)

try:
    seller = bs.find('span', text='Oferta od').parent.a.text.strip()
except:
    seller = ''

print('This is the seller: ', seller)

try:
    mileage = bs.find('span', text='Przebieg').parent.div.text.strip()
except:
    mileage = ''

print('This is the mileage: ', mileage)

try:
    engine_capacity = bs.find('span', text='Pojemność skokowa').parent.div.text.strip()
except:
    engine_capacity = ''

print('This is the engine_capacity: ', engine_capacity)

try:
    fuel_type = bs.find('span', text='Rodzaj paliwa').parent.a.text.strip()
except:
    fuel_type = ''

print('This is the fuel_type: ', fuel_type)

try:
    power = bs.find('span', text='Moc').parent.div.text.strip()
except:
    power = ''

print('This is the power: ', power)

try:
    gearbox = bs.find('span', text='Skrzynia biegów').parent.a.text.strip()
except:
    gearbox = ''

print('This is the gearbox: ', gearbox)

try:
    drive_wheel = bs.find('span', text='Napęd').parent.a.text.strip()
except:
    drive_wheel = ''

print('This is the drive_wheel: ', drive_wheel)

try:
    CO2_emission = bs.find('span', text='Emisja CO2').parent.div.text.strip()
except:
    CO2_emission = ''

print('This is the CO2_emission: ', CO2_emission)

try:
    vehicle_type = bs.find('span', text='Typ').parent.a.text.strip()
except:
    vehicle_type = ''

print('This is the vehicle_type: ', vehicle_type)

# book = {}

# book = {'title':title, 'author':author, 'publisher':publisher, 'category':category, 
#         'language':language, 'score':score, 'ebookPrice':ebookPrice, 
#         'audiobookPrice':audiobookPrice, 'paperPrice':paperPrice, 'popularity':popularity}


# books = books.append(book, ignore_index = True)

# #print(d.to_string(index=False)) #print without indexing

# print(books)

This is the brand:  Volvo
This is the model:  XC 60
This is the version:  D3 R-Design
This is the generation:  I (2008-2017)
This is the year_production:  2011
This is the seller:  Osoby prywatnej
This is the mileage:  165 000 km
This is the engine_capacity:  1 984 cm3
This is the fuel_type:  Diesel
This is the power:  163 KM
This is the gearbox:  Manualna
This is the drive_wheel:  Na przednie koła
This is the CO2_emission:  154 g/km
This is the vehicle_type:  SUV
